In [ ]:
from utility_functions import create_dataset, create_bq_table
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from analysisConfig import cluster_analysis as analysis
from analysisConfig import connection_string, integration_dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import silhouette_score
import gmaps
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier  
import statsmodels.api as sm
from scipy import spatial
from analysisConfig import places_api as api_key
gmaps.configure(api_key=api_key)
from io import StringIO
import psycopg2
import os
import gc
import logging


In [ ]:
query_locations = "/Users/bajajn2/ShortcutScripts/newScripts/"+analysis.dataset

In [ ]:
project = analysis.project
dataset = analysis.dataset
latest_feature_dataset = analysis.feature_dataset

In [ ]:
print(analysis.company)

In [ ]:
dataset = create_dataset(project = project, dataset_id=dataset, confirm_dataset='n')

In [ ]:
create_bq_table(project=project, dataset = dataset, file_location=query_locations, table = 'company_cust', feature_table=latest_feature_dataset, integration_table = integration_dataset)

In [ ]:
create_bq_table(project=project, dataset = dataset, file_location=query_locations, table = 'company_cust_filtered', feature_table=latest_feature_dataset)

In [ ]:
create_bq_table(project=project, dataset = dataset, file_location=query_locations, table = 'custpostcode_service_spending', feature_table=latest_feature_dataset, integration_table=integration_dataset)


In [ ]:
df = pd.read_gbq("""select * from `cluster_analysis.company_cust_filtered` where cmpnyid = {}""".format(analysis.company), project, dialect = 'standard')

In [ ]:
def show_subplots_4(data):
    
    df = data.copy()

    plt.figure(figsize = (20,20))

    ax = plt.subplot(4, 1, 1); sns.distplot(df['Recency'])
    ax.set_xlabel('Recency')
    # Plot frequency distribution
    ax1 = plt.subplot(4, 1, 2); sns.distplot(df['Frequency'])
    ax1.set_xlabel('Frequency')

    # Plot monetary value distribution
    ax2 = plt.subplot(4, 1, 3); sns.distplot(df['Monetary_value'])
    ax2.set_xlabel("Monetary Value")

    #Plot Tenure distriburtion
    ax3 = plt.subplot(4, 1, 4); sns.distplot(df['Tenure'])
    ax3.set_xlabel("Tenure")

    # Show the plot
    plt.show()

In [ ]:
def cluster_stats(data, normalized_data, clusters = 2):
    df = data.copy()
    df_normalized = normalized_data
    kmeans = KMeans(n_clusters=clusters, random_state=1, n_jobs=-1) 

    # Fit k-means clustering on the normalized data set
    kmeans.fit(df_normalized)

    # Extract cluster labels
    cluster_labels = kmeans.labels_
    df_rfm = df.assign(Cluster=cluster_labels)
    df_normalized = df_normalized.assign(Cluster = cluster_labels)

    # Group the data by cluster
    grouped = df_rfm.groupby(['Cluster'])

    # Calculate average RFM values and segment sizes per cluster value
    df_grouped = grouped.agg({
        'Recency': 'mean',
        'Frequency': 'mean',
        'Tenure': 'mean',
        'Monetary_value': ['mean', 'count']
      }).round(1)
    return df_grouped, df_normalized, df_rfm

In [ ]:
df_subset = df[['Recency','Frequency','Monetary_value','Tenure']]
df_log = np.log(df_subset)
scaler = StandardScaler()
scaler.fit(df_log)

# Scale and center the data
df_normalized = scaler.transform(df_log)

# Create a pandas DataFrame
df_normalized = pd.DataFrame(data=df_normalized, index=df_subset.index, columns=df_subset.columns)

In [ ]:
show_subplots_4(df_normalized)

In [ ]:
show_subplots_4(df_subset)

In [ ]:
df_normalized.describe()

In [ ]:
sse = {}
silhouette_average =  []
for k in range(2,8):
    kmeans = KMeans(n_clusters=k, random_state=1, n_jobs=-1)
    kmeans.fit(df_normalized)
    #cluster_labels = kmeans.fit_predict(df_normalized)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    #silhouette_average.append(silhouette_score(df_normalized, cluster_labels,sample_size=20000, random_state=1))
    #print("For n_clusters =", k,
    #      "The average silhouette_score is :", silhouette_average[k-2])
    sse[k] = kmeans.inertia_
    
Optimal_clusters_inertia = np.argmax([k for k in np.abs([j for j in np.diff([i for i in sse.values()])] + [0-[i for i in sse.values()][-1]])/np.array([i for i in sse.values()]) if k < 1]) + 3
#Optimal_clusters_silhoutte = np.argmax(silhouette_average) + 2
print("Optimal number of clusters chosen by k means inertia is {} and by silhoutte distance is not known".format(Optimal_clusters_inertia))

In [ ]:
# Add the plot title "The Elbow Method"
plt.title('The Elbow Method')

# Add X-axis label "k"
plt.xlabel('k')

# Add Y-axis label "SSE"
plt.ylabel('SSE')

# Plot SSE values for each key in the dictionary
sns.pointplot(x=list(sse.keys()), y=list(sse.values()))
plt.show()

In [ ]:
df_grouped = cluster_stats(df, df_normalized, clusters=Optimal_clusters_inertia)
df_normalized = df_grouped[1]
df_melt = pd.melt(df_normalized.reset_index(), 
                    id_vars=['Cluster'],
                    value_vars=['Recency', 'Frequency', 'Monetary_value', 'Tenure'], 
                    var_name='Attribute', 
                    value_name='Value')

In [ ]:
plt.title('Snake plot of standardized variables')
sns.lineplot(x="Attribute", y="Value", hue='Cluster', data=df_melt)

In [ ]:
columns = ["Recency","Frequency","Monetary_value", 'Tenure']
cluster_average = df_grouped[2][columns + ['Cluster']].groupby(['Cluster']).mean()
population_average = df_grouped[2][columns].mean()
relative_importance = cluster_average/population_average - 1
Important_cluster = np.argmax(np.sum(relative_importance[['Frequency','Monetary_value']],1))

In [ ]:
relative_importance = cluster_average/population_average - 1

In [ ]:
plt.figure(figsize=(9.7, 4))
plt.title('cluster average')
sns.heatmap(data=cluster_average, annot=True, fmt='.2f', cmap='RdYlGn')
plt.show()

In [ ]:
plt.figure(figsize=(9.7, 4))
plt.title('Relative importance of attributes')
sns.heatmap(data=relative_importance, annot=True, fmt='.2f', cmap='RdYlGn')
plt.show()

In [ ]:
df_grouped[2]['Cluster'].value_counts()

In [ ]:
df_Important_cluster = df_grouped[2][df_grouped[2].Cluster == Important_cluster]

In [ ]:
def pivot_table_percent(data, agg, index, columns, values, percent = True):
    df = data.copy()
    df_pivot = df.pivot_table(index = index, columns = columns, values = values, aggfunc = agg).reset_index()
    if percent == True:
        df_pivot[values] = [i/df_pivot[values].sum() for i in df_pivot[values]]
    return df_pivot
    

In [ ]:
def change_visualisation(data, datafrom, agg, columns = None, values = None, index = None, percent = True ):
    df = data.copy()
    df1 = datafrom.copy()
    df = pivot_table_percent(df, agg, index, columns, values, percent)
    df1 = pivot_table_percent(df1, agg, index, columns, values, percent)
    df['ratio'] = (df[values]/df1[values] - 1)*100
    plt.figure(figsize=(9.7, 4))
    
    ax = sns.barplot(x = df[index], y = df["ratio"])
    ax.set_title("Relative change in {} of {}".format(agg,index))
    ax.set(ylabel = "ratio %")
    plt.show()

In [ ]:
change_visualisation(df_Important_cluster, df_grouped[2],agg = 'count', values = 'custid', index = "agebin")

In [ ]:
change_visualisation(df_Important_cluster, df_grouped[2],agg = 'count', values = 'custid', index = "cust_state")

In [ ]:
change_visualisation(df_Important_cluster, df_grouped[2],agg = 'count', values = 'custid', index = "cust_gender")

In [ ]:
change_visualisation(df_Important_cluster, df_grouped[2],agg = 'mean', values = 'txnval_total', index = "agebin", percent = False)

In [ ]:
change_visualisation(df_Important_cluster, df_grouped[2],agg = 'mean', values = 'txnval_total', index = "cust_state", percent = False)

In [ ]:
change_visualisation(df_Important_cluster, df_grouped[2],agg = 'mean', values = 'txnval_total', index = "cust_gender", percent = False)

In [ ]:
change_visualisation(df_Important_cluster, df_grouped[2],agg = 'mean', values = 'txncnt_cnt', index = "agebin", percent = False)

In [ ]:
change_visualisation(df_Important_cluster, df_grouped[2],agg = 'mean', values = 'txncnt_cnt', index = "cust_gender", percent = False)

In [ ]:
change_visualisation(df_Important_cluster, df_grouped[2],agg = 'mean', values = 'txncnt_cnt', index = "cust_state", percent = False)

In [ ]:
df_main = df_grouped[2].drop(['Cluster'],1)
df_main['Cluster'] = 'All'
df_final = pd.concat([df_grouped[2],df_main],axis = 0, ignore_index=True) 

In [ ]:
df_final_agebin_mean = df_final.groupby(['Cluster','agebin']).agg({'Recency':'mean','Frequency':'mean','Monetary_value':'mean','Tenure':'mean'}).reset_index()
df_final_agebin_mean['ClusterAgebin'] = [(str(i) + " : " + j) for i,j in zip(df_final_agebin_mean.Cluster,df_final_agebin_mean.agebin)]
df_final_agebin_mean = df_final_agebin_mean.drop(['Cluster','agebin'],1)
df_final_agebin_mean_subset = df_final_agebin_mean[["Recency","Frequency","Monetary_value","Tenure"]]
df_final_agebin_mean_log = np.log(df_final_agebin_mean_subset)
scaler = StandardScaler()
scaler.fit(df_final_agebin_mean_log)

# Scale and center the data
df_final_agebin_mean_normalized = scaler.transform(df_final_agebin_mean_log)


# Create a pandas DataFrame
df_final_agebin_mean_normalized = pd.DataFrame(data=df_final_agebin_mean_normalized, index=df_final_agebin_mean_subset.index, columns=df_final_agebin_mean_subset.columns)
df_final_agebin_mean_normalized = df_final_agebin_mean_normalized.assign(Cluster = df_final_agebin_mean['ClusterAgebin'])
df_final_agebin_melt = pd.melt(df_final_agebin_mean_normalized.reset_index(), 
                    id_vars=['Cluster'],
                    value_vars=['Recency', 'Frequency', 'Monetary_value', 'Tenure'], 
                    var_name='Attribute', 
                    value_name='Value')
plt.figure(figsize=(20,15))
ax = sns.lineplot(x="Attribute", y="Value", hue='Cluster', data=df_final_agebin_melt)
ax.set_title('Snake plot of standardized variables')
ax.legend(bbox_to_anchor=(1.1, 1.05))
plt.show()


In [ ]:
df_final_cust_gender_mean = df_final.groupby(['Cluster','cust_gender']).agg({'Recency':'mean','Frequency':'mean','Monetary_value':'mean','Tenure':'mean'}).reset_index()
df_final_cust_gender_mean['Clustercust_gender'] = [(str(i) + " : " + j) for i,j in zip(df_final_cust_gender_mean.Cluster,df_final_cust_gender_mean.cust_gender)]
df_final_cust_gender_mean = df_final_cust_gender_mean.drop(['Cluster','cust_gender'],1)
df_final_cust_gender_mean_subset = df_final_cust_gender_mean[["Recency","Frequency","Monetary_value","Tenure"]]
df_final_cust_gender_mean_log = np.log(df_final_cust_gender_mean_subset)
scaler = StandardScaler()
scaler.fit(df_final_cust_gender_mean_log)

# Scale and center the data
df_final_cust_gender_mean_normalized = scaler.transform(df_final_cust_gender_mean_log)


# Create a pandas DataFrame
df_final_cust_gender_mean_normalized = pd.DataFrame(data=df_final_cust_gender_mean_normalized, index=df_final_cust_gender_mean_subset.index, columns=df_final_cust_gender_mean_subset.columns)
df_final_cust_gender_mean_normalized = df_final_cust_gender_mean_normalized.assign(Cluster = df_final_cust_gender_mean['Clustercust_gender'])
df_final_cust_gender_melt = pd.melt(df_final_cust_gender_mean_normalized.reset_index(), 
                    id_vars=['Cluster'],
                    value_vars=['Recency', 'Frequency', 'Monetary_value', 'Tenure'], 
                    var_name='Attribute', 
                    value_name='Value')
plt.figure(figsize=(20,15))
ax = sns.lineplot(x="Attribute", y="Value", hue='Cluster', data=df_final_cust_gender_melt)
ax.set_title('Snake plot of standardized variables')
ax.legend(bbox_to_anchor=(1.1, 1.05))
plt.show()


In [ ]:
df_final_cust_state_mean = df_final.groupby(['Cluster','cust_state']).agg({'Recency':'mean','Frequency':'mean','Monetary_value':'mean','Tenure':'mean'}).reset_index()
df_final_cust_state_mean['Clustercust_state'] = [(str(i) + " : " + j) for i,j in zip(df_final_cust_state_mean.Cluster,df_final_cust_state_mean.cust_state)]
df_final_cust_state_mean = df_final_cust_state_mean.drop(['Cluster','cust_state'],1)
df_final_cust_state_mean_subset = df_final_cust_state_mean[["Recency","Frequency","Monetary_value","Tenure"]]
df_final_cust_state_mean_log = np.log(df_final_cust_state_mean_subset)
scaler = StandardScaler()
scaler.fit(df_final_cust_state_mean_log)

# Scale and center the data
df_final_cust_state_mean_normalized = scaler.transform(df_final_cust_state_mean_log)


# Create a pandas DataFrame
df_final_cust_state_mean_normalized = pd.DataFrame(data=df_final_cust_state_mean_normalized, index=df_final_cust_state_mean_subset.index, columns=df_final_cust_state_mean_subset.columns)
df_final_cust_state_mean_normalized = df_final_cust_state_mean_normalized.assign(Cluster = df_final_cust_state_mean['Clustercust_state'])
df_final_cust_state_melt = pd.melt(df_final_cust_state_mean_normalized.reset_index(), 
                    id_vars=['Cluster'],
                    value_vars=['Recency', 'Frequency', 'Monetary_value', 'Tenure'], 
                    var_name='Attribute', 
                    value_name='Value')
plt.figure(figsize=(20,15))
ax = sns.lineplot(x="Attribute", y="Value", hue='Cluster', data=df_final_cust_state_melt)
ax.set_title('Snake plot of standardized variables')
ax.legend(bbox_to_anchor=(1.1, 1.05))
plt.show()


In [ ]:
def change(data, datafrom, agg, columns = None, values = None, index = None, percent = True, top = 100, customers = 50 ):
    df = data.copy()
    df1 = datafrom.copy()
    df = pivot_table_percent(df, agg, index, columns, values, percent)
    df1 = pivot_table_percent(df1, agg, index, columns, values, percent)
    df = pd.merge(df,df1, on = index)
    df['ratio'] = (df[values + '_x']/df[values + '_y'])*100
    if agg == 'count':
        df = df[df[values + '_y'] >= customers]    
    df = df.sort_values('ratio', ascending=False).reset_index().drop(['index'],1)
    df[index] = df[index].astype('int64')
    print(df.shape)
    df_top = df[0:int(df.shape[0]/5)]
    if df_top.shape[0] > top:
        df_top = df_top[0:top]
    df_bottom = df[int((df.shape[0]/5)*4):]
    df_bottom = df_bottom.reset_index().drop(['index'],1)
    if df_bottom.shape[0] > top:
        df_bottom = df_bottom.tail(top)
        df_bottom = df_bottom.reset_index().drop(['index'],1)
    df_remaining = df[(df[index].isin([i for i in df_bottom[index]]) == False) & (df[index].isin([i for i in df_top[index]]) == False) ]
    df_remaining = df_remaining.reset_index().drop(['index'],1)
    return df_top, df_bottom, df_remaining

In [ ]:
df_postcode_hotspots, df_postcode_bottom, df_remaining = change(df_Important_cluster, df_grouped[2], 'count', values = 'custid',index = 'cust_postcode', percent=False, customers = 50 )

In [ ]:
df_remaining[df_remaining.cust_postcode == 2750]

In [ ]:
df_postcode_bottom[df_postcode_bottom.cust_postcode == 2750]

In [ ]:
df_grouped[2][df_grouped[2].cust_postcode == "2750"]

In [ ]:
df_postcode_hotspots

In [ ]:
opportunity_postcodes

In [ ]:
query = """

select postcode, lat, lon
          from
          (
          select postcode as postcode1, sum(total_persons_males + total_persons_females) as total_count
          from
         `dev_external_data.postcode_features_2016`
          group by postcode)

inner join 

`dev_external_data.postcode` on
postcode1 = safe_cast(postcode as int64)
where country_code = "au"


"""

In [ ]:
df_postcode = pd.read_gbq(query, project_id=project, dialect="standard")
df_postcode.postcode = df_postcode.postcode.astype('int64')

In [ ]:
df_postcode_hotspots = pd.merge(df_postcode_hotspots, df_postcode, left_on='cust_postcode', right_on='postcode')

In [ ]:
df_postcode_hotspots.to_csv("C:/Users/bajaj/Desktop/hotspots.csv", index=False)

In [ ]:
# Map centered on London
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(df_postcode_hotspots[['lat','lon']], weights=df_postcode_hotspots.custid_x, max_intensity=100,point_radius=5, opacity=1)
fig.add_layer(heatmap_layer)
fig

In [ ]:
query = """select * from `dev_external_data.postcode_features_2016`"""

In [ ]:
postcode_features = pd.read_gbq(query, project, dialect = "standard")

In [ ]:
query = """select * from `cluster_analysis.custpostcode_service_spending`"""
postcode_spending_features = pd.read_gbq(query, project, dialect = "standard")


postcode_spending_features['service_code'] = ['service_code_'+ str(i) for i in postcode_spending_features['service_code']]


postcode_service_code_txnval_total = pd.pivot_table(postcode_spending_features, columns='service_code', index='cust_postcode', values='txnval_total',fill_value=0).reset_index()
postcode_service_code_txnval_total = postcode_service_code_txnval_total.rename(columns={i:i+'_txnval_total' for i in postcode_service_code_txnval_total if 'service_code' in i})
postcode_service_code_txncnt_cnt = pd.pivot_table(postcode_spending_features, columns='service_code', index='cust_postcode', values='txncnt_cnt',fill_value=0).reset_index()
postcode_service_code_txncnt_cnt = postcode_service_code_txncnt_cnt.rename(columns={i:i+'_txncnt_cnt' for i in postcode_service_code_txncnt_cnt if 'service_code' in i})
postcode_service_code_custcnt_cnt = pd.pivot_table(postcode_spending_features, columns='service_code', index='cust_postcode', values='custcnt_cnt',fill_value=0).reset_index()
postcode_service_code_custcnt_cnt = postcode_service_code_custcnt_cnt.rename(columns={i:i+'_custcnt_cnt' for i in postcode_service_code_custcnt_cnt if 'service_code' in i})



postcode_service_code_features = pd.merge(postcode_service_code_custcnt_cnt, postcode_service_code_txnval_total, on = 'cust_postcode', how = 'inner')
postcode_service_code_features = pd.merge(postcode_service_code_features, postcode_service_code_txncnt_cnt, on = 'cust_postcode', how='inner')
scaler = MinMaxScaler()
postcode_X = postcode_service_code_features[[i for i in postcode_service_code_features if 'service_code' in i]]
scaler.fit(postcode_X)
postcode_normalised = scaler.transform(postcode_X)
postcode_service_code_features_normalised = pd.DataFrame(data=postcode_normalised, index = postcode_X.index, columns = postcode_X.columns)
postcode_service_code_features_normalised = postcode_service_code_features_normalised.assign(cust_postcode = postcode_service_code_features.cust_postcode)

In [ ]:
#postcode_service_code_features.to_gbq('cluster_analysis.postcode_service_code_features', 'anz-insto-data-analytics_dev',if_exists='replace' )
#postcode_service_code_features_normalised.to_gbq('cluster_analysis.postcode_service_code_features_normalised', 'anz-insto-data-analytics_dev',if_exists='replace' )

In [ ]:
postcode_pct_features = [i for i in postcode_features if 'pct' in i]
postcode_dwelling_features = [i for i in postcode_features if "dwelling" in i]
postcode_features = postcode_features[postcode_features.total_persons_males + postcode_features.total_persons_females >= 1000].reset_index().drop(["index"],1)
columns = ["postcode"] + postcode_pct_features + postcode_dwelling_features
postcode_features = postcode_features[columns]
postcode_features_subset = postcode_features[postcode_dwelling_features]
scaleminmax = MinMaxScaler()
scaleminmax.fit(postcode_features_subset)
postcode_features_scaled = scaleminmax.transform(postcode_features_subset)
postcode_features_scaled = pd.DataFrame(postcode_features_scaled, index = postcode_features_subset.index,columns = postcode_features_subset.columns )
postcode_features = postcode_features[["postcode"] + postcode_pct_features]
postcode_features = postcode_features[[ i for i in postcode_features if postcode_features[i].dtype != "O"]]
for i in postcode_features:
    if "pct" in i:
        #print(i)
        postcode_features[i] = list(map(lambda x: x/100 if x > 1 else x, postcode_features[i]))
postcode_features = pd.concat([postcode_features, postcode_features_scaled], axis = 1)
postcode_features["hotspot"] = list(map(lambda x : 1 if x in [i for i in df_postcode_hotspots.cust_postcode] else 0,[j for j in postcode_features.postcode])) 
postcode_service_code_features_normalised = postcode_service_code_features_normalised.rename(columns = {'cust_postcode':'postcode'})
postcode_service_code_features_normalised.postcode = postcode_service_code_features_normalised.postcode.astype(int)
postcode_features = pd.merge(postcode_service_code_features_normalised,postcode_features, how = 'inner', on = 'postcode')
TrainingColumns = [i for i in postcode_features if i not in ["postcode","hotspot"]]
X = postcode_features[TrainingColumns]
y = postcode_features["hotspot"]
postcode_features_Test = postcode_features[postcode_features.hotspot == 0]
postcode_features_Test = postcode_features_Test.reset_index().drop(['index'],1)
postcode_features_Test_Predict = postcode_features_Test[TrainingColumns]

In [ ]:
def model_probability(TrainX,Trainy, TestX, TestDataframe, model = None, top = 100):
    if model == "rfc":
        m = RandomForestClassifier(random_state=1,n_jobs=-1, class_weight='balanced')
        m.fit(TrainX,Trainy)
        TestDataframe["Probability"] = [i[1] for i in m.predict_proba(TestX) ]
        TestDataframe = TestDataframe.sort_values('Probability', ascending = False).reset_index().drop(["index"],1)[0:top]
    elif model == "lr":
        m = LogisticRegression(class_weight='balanced', random_state=1)
        m.fit(TrainX,Trainy)
        logit_model=sm.Logit(Trainy,TrainX)
        result=logit_model.fit(method='bfgs')
        print(result.summary2())

        TestDataframe["Probability"] = [i[1] for i in m.predict_proba(TestX) ]
        feature_importance = abs(m.coef_[0])
        feature_importance = 100.0 * (feature_importance / feature_importance.max())
        #feature_importance
        
        TestDataframe = TestDataframe[TestDataframe.Probability >= 0.8]  
        TestDataframe = TestDataframe.sort_values('Probability', ascending = False).reset_index().drop(["index"],1)  
        
        print([(i ,ind) for ind, i in enumerate(feature_importance) if i > 50])
        sorted_idx = np.argsort(feature_importance)
        #print(sorted_idx)
        pos = np.arange(sorted_idx.shape[0]) + .5

        featfig = plt.figure(figsize = (20,10))
        featax = featfig.add_subplot(1, 1, 1)
        featax.barh(pos, feature_importance[sorted_idx], align='center')
        featax.set_yticks(pos)
        featax.set_yticklabels(np.array(X.columns)[sorted_idx], fontsize=8)
        featax.set_xlabel('Relative Feature Importance')

        plt.tight_layout()   
        plt.show()
        
        
    TestDataframe.Probability = [format(i,'f') for i in TestDataframe.Probability]
    return TestDataframe

In [ ]:
model_probability(X,y, postcode_features_Test_Predict, postcode_features_Test, model = 'rfc')

In [ ]:
#model_probability(X,y, postcode_features_Test_Predict, postcode_features_Test, model = 'lr')

In [ ]:
def cosine_similarity(l1, l2):
    result = 1 - spatial.distance.cosine(l1, l2)
    return result

In [ ]:
l1 = {}
l2 = {}
l3 = {}
for i in postcode_features['postcode']:
    if i in [j for j in df_postcode_hotspots.cust_postcode]:
        l1[i] = postcode_features[postcode_features['postcode'] == i][[j for j in X.columns]].values.tolist()   
    else:
        if i in [j for j in df_postcode_bottom.cust_postcode]:
            l2[i] = postcode_features[postcode_features['postcode'] == i][[j for j in X.columns ]].values.tolist()   
        elif i in [j for j in df_remaining.cust_postcode]:
            l3[i] = postcode_features[postcode_features['postcode'] == i][[j for j in X.columns ]].values.tolist()


In [ ]:
postcode1 = {}
for k in l3:
    scores1 = []
    scores2 = []
    postcode2 = []
    postcode3 = []
    for j in l2:
        scores1.append(cosine_similarity(l3[k],l2[j]))
        postcode2.append(j)
    for i in l1:
        scores2.append(cosine_similarity(l3[k],l1[i]))
        postcode3.append(i)
    if np.max(scores1) < np.max(scores2):
        postcode1[k] = postcode3[np.argmax(scores2)]

In [ ]:
opportunity_postcodes = pd.DataFrame.from_dict(postcode1, 'index').reset_index().rename(columns = {'index':'postcode',0:'SimilarPostcode'})
df_postcode.postcode = df_postcode.postcode.astype('int64')
opportunity_postcode_hotspots = pd.merge(opportunity_postcodes, df_postcode, on = 'postcode')
opportunity_postcode_hotspots = opportunity_postcode_hotspots.reset_index().drop(['index'],1)

In [ ]:
fig = gmaps.figure(center=(-37.8102, 144.96), zoom_level=5)
marker_layer = gmaps.marker_layer(opportunity_postcode_hotspots[['lat','lon']], info_box_content= ['postcode:' + str(i) for i in opportunity_postcode_hotspots.postcode], hover_text=['Similar to postcode: ' + str(i) for i in opportunity_postcode_hotspots['SimilarPostcode']])
marker_layer_2 = gmaps.symbol_layer(df_postcode_hotspots[['lat','lon']], info_box_content=['postcode:' + str(i) for i in df_postcode_hotspots.postcode])
fig.add_layer(marker_layer)
fig.add_layer(marker_layer_2)
fig

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=5, weights='distance')  
top_postcode = postcode_features[postcode_features.postcode.isin([i for i in l1])].reset_index().drop(['index','hotspot'],1).assign(Label = 1)
bottom_postcode = postcode_features[postcode_features.postcode.isin([i for i in l2])].reset_index().drop(['index','hotspot'],1).assign(Label = -1)
remaining_postcode = postcode_features[postcode_features.postcode.isin([i for i in l3])].reset_index().drop(['index','hotspot'],1)
postcode_ranking = pd.concat([top_postcode, bottom_postcode], axis=0, ignore_index=True)
X = postcode_ranking[TrainingColumns]
y = postcode_ranking['Label']

In [ ]:
classifier.fit(X,y)

In [ ]:
remaining_postcode

In [ ]:
Test = remaining_postcode[TrainingColumns]
remaining_postcode['Label'] = classifier.predict(Test)

In [ ]:
remaining_postcode = remaining_postcode[remaining_postcode.Label == 1].reset_index().drop(['index'],1)

In [ ]:
remaining_postcode

In [ ]:
df_remaining = df_remaining.rename(columns = {'cust_postcode':'postcode'})
df_remaining_postcode  = pd.merge(df_remaining, remaining_postcode, on = 'postcode')
final_postcode = pd.merge(df_remaining_postcode, df_postcode, on = "postcode")

In [ ]:
fig = gmaps.figure(center=(-37.8102, 144.96), zoom_level=5)
marker_layer = gmaps.marker_layer(final_postcode[['lat','lon']], info_box_content= ['postcode:' + str(i) + " coverts " + str(j) + " top segment customers out of " + str(k) for i,j,k in zip(final_postcode.postcode, final_postcode.custid_x, final_postcode.custid_y)])
fig.add_layer(marker_layer)
fig

In [ ]:
df_remaining_postcode.to_csv("C:/Users/bajaj/Desktop/opportunity_postcode.csv", index=False)

In [ ]:
company_top_postcodes = df_postcode_hotspots[['cust_postcode','ratio','lat','lon']]
company_opportunity_postcodes = final_postcode[['postcode','ratio','lat','lon']]
company_top_postcodes['cmpnyid'] = analysis.company
company_opportunity_postcodes['cmpnyid'] = analysis.company

In [ ]:
company_opportunity_postcodes

In [ ]:
"""This will create a dictionary of data types from BQ to Postgres"""
def data_types(dataframe):
    df = dataframe.copy()
    ul = [str(i) for i in df.dtypes]
    datatypes = list(map(lambda x: " INTEGER" if 'int' in x else " DECIMAL" if 'float' in x else " TEXT" if 'object' in x else " VARCHAR(255)", ul))
    return datatypes

In [ ]:
def write_data_to_pg(data, tablename, connection_string = None, schema = ""):
    df = data.copy()
    sio = StringIO()
    sio.write(df.to_csv(index=None, header=None))
    sio.seek(0)
    file_object = sio
    connection = psycopg2.connect(connection_string)
    destination = "{}.{}".format(schema, tablename)
    datatypes = data_types(df)
    columns = ", ".join([(i + " " + j) for i,j in zip(df.columns, datatypes)])
    schema_command = """create schema IF NOT EXISTS {};""".format(schema)
    cur = connection.cursor()
    cur.execute(schema_command)
    connection.commit()
    command = """create table IF NOT EXISTS {}({});""".format(destination,columns)
    logging.info(command)
    cur = connection.cursor()
    cur.execute(command)
    connection.commit()
    with connection.cursor() as c:
        columns = ', '.join([f'{col}' for col in df.columns])
        sql = f'COPY {destination} ({columns}) FROM STDIN WITH CSV'
        c.copy_expert(sql=sql, file=file_object)
        connection.commit()
        c.close()
        del c
    return logging.info("transfer successful for table {}".format(destination))

In [ ]:
write_data_to_pg(company_top_postcodes,tablename='company_top_postcodes', connection_string=connection_string, schema='feature')
write_data_to_pg(company_opportunity_postcodes,tablename='company_opportunity_postcodes', connection_string=connection_string, schema='feature')